# Imports

In [1]:
!pip install youtube_dl
!pip install pytube
!git clone https://github.com/XniceCraft/ffmpeg-colab.git
!chmod +x ./ffmpeg-colab/install
!./ffmpeg-colab/install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 33.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 5.3 MB/s 
Cloning into 'ffmpeg-colab'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 28 (delta 6), reused 0 (delta 0), pack-reused 14
Unpacking objects: 100% (28/28), done.
FFmpeg was found at /usr/bin/ffmpeg
Removing old FFmpeg
Done
Moving new ffmpeg to /usr/bin
FFmpeg was successfully installed


In [2]:
import requests
import os, sys
import urllib.request
import re
import subprocess
import shlex
import random
from datetime import datetime
from pytube import YouTube

# Downloading

In [3]:
def downloadBackground(URLs):
  for url in URLs:
    yt = YouTube(url)
    title = yt.title.replace(" ","-")
    if os.path.exists('/content/' + title + '.mp4'):
      print("Background " + title + " --> Already Downloaded")
    else:
      video_stream = yt.streams.filter(file_extension='mp4').get_highest_resolution()
      video_stream.download(filename=title + '.mp4')
      print("Background " + title + " --> Successfully Downloaded")

In [4]:
def downloadMP4FromURL(video_url,video_name):
  # Send an HTTP GET request to the URL
  response = requests.get(video_url)

  # Check the status code of the response
  if response.status_code == 200:
    # Write the content of the response to a file
    dir = '/content/' + video_name.split('.mp4')[0]
    os.makedirs(dir)
    with open(dir + '/' + video_name, 'wb') as f:
        f.write(response.content)
        print('Video Succesfully Downloaded')
  else:
    print('Failed to download video')


In [5]:
def getDOMTree(link_url,video_name):

  if os.path.exists('/content/' + video_name.split('.mp4')[0]):
    print('Video Already Downloaded')
  else:
    #HAVE TO MASK AS USER SO DOMAIN DOESN'T RECOGNIZE WEB SCRAPER
    HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
    r = requests.get(link_url,headers=HEADERS)
    index = r.text.find('contentUrl')

    if index != -1:
      # Get the line that contains the string by slicing the text
      line = r.text[index:].split('\n')[0]
      match = re.search(r'contentUrl": "(.+?)"', line)

      # Check if a match was found
      if match:
        # Get the URL from the match object
        url = match.group(1)
        downloadMP4FromURL(url,video_name)
      else:
        print('URL not found')
    else:
      print('String not found')

In [6]:
def fix_timestamp(timestamp):
  # Use a regular expression to check if the timestamp starts with '00:'
  if not re.match(r'^00:', timestamp):
    # If the timestamp does not start with '00:', add it to the beginning
    timestamp = '00:' + timestamp
  return timestamp + '.000'

# Cropping

In [8]:
def cropVideo(video_name):

  backgroundPath = '/content/ForzaHorizon4DRIVINGLIKEABOSS!!1876HPRTRMustang.mp4'

  outputPath = '/content/' + video_name.split('.mp4')[0]
  inputFile = outputPath + '/' + video_name

  startTime = input("Enter Start Timestamp: ")
  endTime = input("Enter End Timestamp: ")
  fileName = startTime + '-' + endTime + '.mp4'

  startTime = fix_timestamp(startTime)
  endTime = fix_timestamp(endTime)

  command = ['ffmpeg', '-i', inputFile, '-copyinkf', '-ss', startTime, '-to', endTime, f'{outputPath}/{fileName}']
  subprocess.run(command)

  backgroundLength = getSeconds(getVideoLength(backgroundPath)[0])
  vidLength = getSeconds(getVideoLength(outputPath + '/' + fileName)[0])
  
  backgroundStart = getRandomNumber(30, backgroundLength - vidLength - 30)

  print(backgroundLength,vidLength,backgroundStart)
  command = ['ffmpeg', '-hwaccel', 'cuda', '-i', backgroundPath, '-copyinkf', '-ss', getHHMMSS(backgroundStart), '-to', getHHMMSS(backgroundStart + vidLength), '/content/sample.mp4']
  subprocess.run(command)

  getMutedBackground('/content/sample.mp4')

In [16]:
backgroundPath = '/content/Forza-Horizon-4-DRIVING-LIKE-A-BOSS!!-1876HP-RTR-Mustang.mp4'
command = ['ffmpeg', '-hwaccel', 'cuda', '-i', backgroundPath, '-copyinkf', '-ss', getHHMMSS(30), '-to', getHHMMSS(40), '/content/sample.mp4']
subprocess.run(command)

CompletedProcess(args=['ffmpeg', '-hwaccel', 'cuda', '-i', '/content/Forza-Horizon-4-DRIVING-LIKE-A-BOSS!!-1876HP-RTR-Mustang.mp4', '-copyinkf', '-ss', '00:00:30.000', '-to', '00:00:40.000', '/content/sample.mp4'], returncode=0)

In [ ]:
!ffmpeg -hwaccel cuda -i 

In [7]:
def getVideoLength(video_path):
  command = "ffmpeg -i " + video_path + " 2>&1 | grep 'Duration' | cut -d ' ' -f 4 | sed s/,//"
  try:
    stdout = !{command}
    return stdout
  except:
    print("getVideoLength failed")
  # Get the duration of the video from the command output

In [9]:
def getMutedBackground(video_path):
  muted_video_path = video_path.split('.mp4')[0] + 'Muted.mp4'
  command = "ffmpeg -i " + video_path + " -c copy -an -y " + muted_video_path
  try:
    !{command}
  except:
    print("getMutedBackground failed")

In [10]:
def getSeconds(timestamp):
  dt = datetime.strptime(timestamp, "%H:%M:%S.%f")
  return dt.hour*3600 + dt.minute*60 + dt.second

In [11]:
def getHHMMSS(timestamp):
  dt = datetime.utcfromtimestamp(int(timestamp))
  return dt.strftime('%H:%M:%S') + '.000'

In [12]:
def getRandomNumber(a, b):
  # Get a random integer between a and b (inclusive)
  return random.randint(a, b)

In [13]:
#@title Video Game Backgrounds
URLs = ['https://youtu.be/9NgBLirYQS8']
downloadBackground(URLs)

Background Forza-Horizon-4-DRIVING-LIKE-A-BOSS!!-1876HP-RTR-Mustang --> Successfully Downloaded


In [64]:
#@title URL Input
# https://odysee.com/@tatespeech:c/superbike-vs-supercar:1
link_url = input("Enter Odysee Link URL: ")
video_name = link_url.rsplit('/',1)[-1] + '.mp4'

Enter Odysee Link URL: https://odysee.com/@tatespeech:c/superbike-vs-supercar:1


In [95]:
#@title User Actions From URL Input
getDOMTree(link_url,video_name)
cropVideo(video_name)

Video Already Downloaded
Enter Start Timestamp: 02:30
Enter End Timestamp: 02:40
369 10 240
ffmpeg version 5.0 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/content/build --pkg-config-flags=--static --extra-cflags=-I/content/build/include --extra-ldflags=-L/content/build/lib --extra-libs='-lpthread -lm -lgomp' --ld=g++ --bindir=/content/build/bin --disable-ffplay --disable-ffprobe --disable-doc --disable-htmlpages --disable-manpages --disable-podpages --disable-txtpages --enable-version3 --enable-gpl --enable-nonfree --enable-opencl --enable-opengl --disable-shared --enable-static --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfdk-aac --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libkvazaar --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-libshine --enable-lib